# Advanced Querying Mongo

Importing libraries and setting up connection

In [4]:
from pymongo import MongoClient
client = MongoClient()
client.ironhack

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [16]:
db = client.ironhack

In [17]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [18]:
compi = db.compi

In [19]:
compi

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'compi')

In [11]:
# imported data from MongoDB Compass. 
# alternative was: mongoimport --db ironhack --collection companies --file companies.json

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [ ]:
# Your Code

In [21]:
list(compi.find({'name':'Babelgum'}, {'_id': 0, 'name': 1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [ ]:
# Your Code

In [35]:
q = {'number_of_employees':{'$gt': 5000}}
pj = {"_id":0,"name":1}
#sort = {"number_of_employees":1}
res = compi.find(q,pj).sort("number_of_employees",-1).limit(20)
list(res)[:3]

[{'name': 'Siemens'}, {'name': 'IBM'}, {'name': 'Toyota'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [37]:
# Your Code
q = {"founded_year":{"$gte":2000, "$lte":2005}}
pj = {"_id":0,"name":1,"founded_year":1}
res=compi.find(q,pj)
list(res[:4])

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [39]:
# Your Code
q = {"$and": [{"ipo.valuation_amount":{"$gte":100_000_000}},{"founded_year": {"$lt": 2010}}]}
pj = {'name':1, 'ipo.valuation_amount':1, '_id':0}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [ ]:
# Your Code

In [40]:
q = {"$and": [{"number_of_employees": {"$lt": 1000}}, {"founded_year":{"$lt": 2005}}]}
pj = {"_id":0, "name":1, "number_of_employees":1, "founded_year":1}
res = compi.find(q,pj).sort("number_of_employees",-1).limit(10)
list(res[:3])

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997}]

### 6. All the companies that don't include the `partners` field.

In [44]:
# Your Code
res = compi.count_documents({"partners": {"$exists":"0"}})
res

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [45]:
# Your Code
q = {"category_code": {"$type": "null" }}
pj = {"_id":0,"name":1}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'Collective'}, {'name': 'Snimmer'}, {'name': 'KoolIM'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [46]:
# Your Code
q = {"number_of_employees": {"$gt":100,"$lt": 1000}}
pj = {"_id":0,"name":1,"number_of_employees": 1}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305}]

### 9. Order all the companies by their IPO price in a descending order.

In [50]:
# Your Code
q = {"ipo.valuation_amount":{"$exists":1}}
pj = {"_id":0,"name":1, "ipo.valuation_amount":1}
res = compi.find(q,pj).sort("ipo.valuation_amount",-1)
list(res[:3])

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [52]:
# Your Code
q = {"number_of_employees":{"$exists":1}}
pj = {"_id":0,"name":1, "number_of_employees": 1}
res = compi.find(q,pj).sort("number_of_employees",-1).limit(10)
list(res[:3])

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [53]:
# Your Code
q = {"founded_month":{"$gte": 6, "$lt": 12}}
pj = {"_id":0,"name":1}
res = compi.find(q,pj).limit(1000)
list(res[:3])

[{'name': 'Wetpaint'}, {'name': 'Zoho'}, {'name': 'Digg'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [54]:
# Your Code
q = {'$and':[{"founded_year":{"$lt": 2000},"acquisition.price_amount": {'$gt': 10_000_000}}]}
pj = {"_id":0,"name":1}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'Postini'}, {'name': 'SideStep'}, {'name': 'Recipezaar'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [56]:
# Your Code
q = {'acquisitions.acquired_year': {'$gt': 2010}}
pj = {'name':1,'_id':0, 'acquisition':1}
res = compi.find(q,pj).sort('acquisition.price_amount', -1)
list(res[:3])

[{'name': 'LSI',
  'acquisition': {'price_amount': 6600000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://dealbook.nytimes.com/2013/12/16/avago-to-buy-lsi-for-6-6-billion/?_r=0',
   'source_description': 'Avago to Buy LSI for $6.6 Billion',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Avago Technologies',
    'permalink': 'avago-technologies'}}},
 {'name': 'Sabre',
  'acquisition': {'price_amount': 4300000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://venturebeat.com/2006/12/12/sabre-holdings-travel-reservation-co-sold-to-private-equity-firms-for-43-billion/',
   'source_description': 'Sabre Holdings, travel reservation co., sold to private equity firms for $4.3 billion',
   'acquired_year': 2006,
   'acquired_month': 12,
   'acquired_day': None,
   'acquiring_company': {'name': 'Silver Lake Partners & Texas Pacific Group',
    'permalink': 'silver

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [58]:
# Your Code
q = {"founded_year":{"$exists": 1}}
pj = {"_id":0,"name":1,"founded_year":1}
res = compi.find(q,pj).sort("founded_year",-1)
list(res[:3])

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [60]:
# Your Code
q = {"founded_day":{"$lte": 7}}
pj = {"_id":0,"name":1,'acquisition.price_amount':1}
res = compi.find(q,pj).sort('acquisition.price_amount', -1).limit(10)
list(res[:3])

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [63]:
# Your Code
q = {"$and": [{"category_code": "web"},{"number_of_employees": {"$gt":4000}}]}
pj = {"_id":0,"name":1,"number_of_employees":1}
res = compi.find(q,pj).sort("number_of_employees",1)
list(res[:3])

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [65]:
# Your Code
q = {"$and": [{"acquisition.price_amount": {'$gt': 10_000_000}},{"acquisition.price_currency_code":"EUR"}]}
pj = {"_id":0,"name":1,"acquisition.price_amount":1}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [72]:
# Your Code
q = {"acquisition.acquired_month" : {"$gte":1,"$lte":3}}
pj = {"_id":0,"name":1,"acquisition":1}
res = compi.find(q,pj).limit(10)
list(res[:3])

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [74]:
# Your Code
q = {"$and": [{"founded_year":{"$gte":2000, "$lte":2010}},{"acquisition.acquired_year" : {"$gte":2011}}]}
pj = {"_id":0,"name":1,"founded_year":1,"acquisition.acquired_year":1}
res = compi.find(q,pj)
list(res[:3])

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code


In [75]:
q = {"deadpooled_year": {"$gt":{"founded_year":{"founded_year": + 3}}}}
pj = {"_id":0,"name":1,"founded_year":1, "deadpooled_year":1}
res = compi.find(q,pj)
list(res)

[]